<a href="https://colab.research.google.com/github/xunzxe/first-repository/blob/master/Datathon/JK_olympics_data_by_events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#이번에 어떤 이벤트에서 메달 획득할 수 있을까?
* 신체적 차이가 두드러지는 특정 종목에 더 특화된 모델

개최국 여부, 출전 연도, 선수 신체 정보(신장, 체중)를 바탕으로 종목당 메달 획득 가능성 예측 모델



* (개최국 베네핏 회귀모델로 추출한 개최국에 대한 베네핏을 이 모델에 적용합니다.
  * 개최국 범주를 임시로 추가하고(아닙니다. 이벤트별 개최지 정보 참조를 위해 고정으로 추가합니다), 이에 따른 개최국별 베네핏을 새로운 범주로 추가합니다.
  * 출신선수가 개최국 출신인 경우, 개최국 베네핏에 대한 패널티를 적용해 새로운 범주로 추가합니다. / 기존의 실제 메달 획득률은 획득한 메달에 대한 점수를 0~1로 나눈 값으로 인코딩합니다.
    * 추가된 범주는 개최국 베네핏으로 곱하였을 때 실제 메달 획득률 회복)

* (하지 않습니다:) 이벤트명과 개최국명을 샘플데이터로 지정하지 않습니다. 대신 이벤트명의 토큰들을 범주로 추가합니다.
  * 이벤트별 개최지 범주를 새로운 칼럼으로 추가합니다. (패널티 적용 여부 판단에 참조) / 또는 이벤트 이름을 토큰화 (종목, 숫자, 참여자 성별 토큰)하고 범주에 추가합니다.
* 이벤트를 정렬해 인코딩하되, 메달 획득자의 신장 대비 BMI의 내림차순으로 정렬하고 1~0사이의 값으로 인코딩합니다.
* 이벤트별로 속한 종목을 인코딩 (종목을 범주에 추가, 원핫인코딩)합니다.
* 이벤트별 남녀 범주를 구분해 원핫인코딩합니다.


---
* 랜덤포레스트 사용해 회귀모델 작성하는 경우 연도 데이터는 drop
  * (개최국 베네핏 산정 시) 역사적으로 베네핏 변화량에 유의미한 흐름이 있다면 유의미한 변화량 수치화, 범주로 추가
  * (이벤트별, 종목별 회귀모델) 종목별로 유의미한 수치적 변화가 있다면 역시 시간을 반영한 범주로 추가

* 유의미한 변화량을 시계적으로 예측하고자 한다면 고려할 모델들
  * ARIMA
  * SARIMA
  * Prophet
  * LSTM

가능한 다른 방향
(데이터 분석 내용)
* GDP 정보를 추가해 국가별 GDP와 메달 획득 확률 연관성 확인
* 인구 정보를 추가해 국가별 인구수와 참가자수, 메달 획득자 수 수치 비교
* 더 자세한 선수 신체 정보 (체질량 지수 등)
* 경도위도별 기후 정보를 추가해 동하계의 메달획득과 연관이 있는지 확인
*

###Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
import os

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# 지도
!pip install folium
import folium
from folium import Marker
from folium import plugins
from folium import GeoJson

In [ ]:
from sklearn.ensemble import RandomForestClassifier

##STEP1. 데이터 준비

###데이터 불러오기

* athlete_events.csv
* noc_regions.csv

* 추가 데이터
  * 해당 기간 국가별 GDP
  * 경도 위도

In [ ]:
original_regions = pd.read_csv('/content/noc_regions.csv')

In [ ]:
original_athlete = pd.read_csv('/content/athlete_events.csv')

In [ ]:
long_lat = pd.read_csv('/content/coorinates based on the noc regions.csv')

In [ ]:
# original_lin = pd.read_csv('/content/athelete_events_with_L_l_N.csv')

In [ ]:
# original_linc = pd.read_csv('/content/athelete_events_with_L_l_N_C.csv')

In [ ]:
original_hosts = pd.read_csv('/content/olympic_hosts.csv')

복사해서 저장

In [ ]:
athlete = original_athlete.copy()
regions = original_regions.copy()

In [ ]:
longlat = long_lat.copy()
longlat.head()

,notes,latitude,longitude,NOC,region
0,Antigua\ and\ Barbuda Australasia Bohemia Cret...,continuous,continuous,AFG AHO ALB ALG AND ANG ANT ANZ ARG ARM ARU AS...,Afghanistan Albania Algeria American\ Samoa An...
1,meta,meta,meta,NaN,NaN
2,NaN,33.9328,65.181,AFG,Afghanistan
3,Netherlands Antilles,12.213,-69.0308,AHO,Curacao
4,NaN,41.1253,20.0097,ALB,Albania


In [ ]:
# lin = original_lin.copy()
# lin.head()

In [ ]:
# linc = original_linc.copy()
# linc.head()

In [ ]:
# linc_longlatnation = linc[['Nation', 'Latitude', 'Longitude']]
# linc_longlatnation.head()

In [ ]:
# merged_lin = pd.merge(lin, linc_longlatnation, on='Nation' , how='left')
# merged_lin.head()

In [ ]:
host = original_hosts.copy()
print(host.shape)
host.head()

(63, 11)


,Type,GamesUrl,Disciplines,DisciplinesList,Country,Date,Athletes,Countries,Events,City,Year
0,summergames,https://www.olympic.org/athens-1896,10,"['Athletics', 'Cycling Road', 'Cycling Track',...",Greece,06 Apr - 15 Apr,241.0,14.0,43.0,Athens,1896
1,summergames,https://www.olympic.org/paris-1900,20,"['Archery', 'Athletics', 'Basque Pelota', 'Cri...",France,14 May - 28 Oct,997.0,24.0,95.0,Paris,1900
2,summergames,https://www.olympic.org/st-louis-1904,19,"['Archery', 'Athletics', 'Basketball', 'Boxing...",United States of America,01 Jul - 23 Nov,651.0,12.0,95.0,St Louis,1904
3,summergames,https://www.olympic.org/london-1908,25,"['Archery', 'Athletics', 'Boxing', 'Cycling Tr...",Great Britain,27 Apr - 31 Oct,2008.0,22.0,110.0,London,1908
4,summergames,https://www.olympic.org/stockholm-1912,18,"['Athletics', 'Cycling Road', 'Diving', 'Eques...",Sweden,05 May - 27 Jul,2407.0,28.0,102.0,Stockholm,1912


###데이터 확인

두 데이터 프레임 각각의 :
* 모양 확인
* 결측치 확인

####athlete 데이터프레임
* 결측치 칼럼 비율 확인
* 포괄 연도수 확인

#####결측치 비율

In [ ]:
print(athlete.shape)
athlete.head()

(271116, 15)


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [ ]:
athlete.isnull().sum()

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64

In [ ]:
# Null percentage
all = athlete.shape[0]

# Age is null
print('연령 결측 비율 :', athlete['Age'].isnull().sum() / all * 100)

# Height is null
print('신장 결측 비율 :', athlete['Height'].isnull().sum() / all * 100)

# Weight is null
print('체중 결측 비율 :', athlete['Weight'].isnull().sum() / all * 100)


# Medal is not null
print()
print('메달 획득 비율 :', athlete['Medal'].notnull().sum() / all * 100)

연령 결측 비율 : 3.49444518213606
신장 결측 비율 : 22.193821095029435
체중 결측 비율 : 23.19118015904631

메달 획득 비율 : 14.673792767671404


In [ ]:
athlete[athlete['Age'].isna()]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
147,54,Mohamed Jamshid Abadi,M,NaN,NaN,NaN,Iran,IRI,1948 Summer,1948,Summer,London,Boxing,Boxing Men's Heavyweight,NaN
152,58,Georgi Abadzhiev,M,NaN,NaN,NaN,Bulgaria,BUL,1924 Summer,1924,Summer,Paris,Cycling,"Cycling Men's Road Race, Individual",NaN
153,58,Georgi Abadzhiev,M,NaN,NaN,NaN,Bulgaria,BUL,1924 Summer,1924,Summer,Paris,Cycling,"Cycling Men's Road Race, Team",NaN
162,66,Mohamed Abakkar,M,NaN,156.0,48.0,Sudan,SUD,1972 Summer,1972,Summer,Munich,Boxing,Boxing Men's Flyweight,NaN
212,102,Sayed Fahmy Abaza,M,NaN,NaN,NaN,Egypt,EGY,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270679,135368,Antonio A. Zucchi,M,NaN,NaN,NaN,Argentina,ARG,1948 Summer,1948,Summer,London,Hockey,Hockey Men's Hockey,NaN
270774,135416,Star Zulu,M,NaN,NaN,NaN,Zambia,ZAM,1984 Summer,1984,Summer,Los Angeles,Boxing,Boxing Men's Bantamweight,NaN
270793,135426,Max Zumstein,M,NaN,NaN,NaN,Switzerland,SUI,1928 Summer,1928,Summer,Amsterdam,Hockey,Hockey Men's Hockey,NaN
270804,135434,Edmundo Ziga Erraz,M,NaN,180.0,95.0,Chile,CHI,1948 Summer,1948,Summer,London,Athletics,Athletics Men's Hammer Throw,NaN


In [ ]:
# len(athlete[athlete['Weight'].isnull()] & athlete[athlete['Height'].isnull()])

both_na = athlete[(athlete['Weight'].isnull()) & (athlete['Height'].isnull())].shape[0]
print(both_na)

len(athlete[(athlete['Weight'].isnull()) | (athlete['Height'].isnull())])

58783


64263

* Weight, Height 에서 모두 날려버릴 순 없겠다.
* 결측 값을 제외하고 연산

####merge

* NOC 기준으로 국가 칼럼 더하기

In [ ]:
merged = pd.merge(athlete, regions, on='NOC' , how='left')
merged.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN


In [ ]:
set(merged['notes'])

{'Antigua and Barbuda',
 'Australasia',
 'Bohemia',
 'Crete',
 'Hong Kong',
 'Individual Olympic Athletes',
 'Netherlands Antilles',
 'Newfoundland',
 'North Borneo',
 'North Yemen',
 'Refugee Olympic Team',
 'Serbia and Montenegro',
 'South Yemen',
 'Trinidad and Tobago',
 'Turks and Caicos Islands',
 'Tuvalu',
 'United Arab Republic',
 'Unknown',
 'Virgin Islands',
 'West Indies Federation',
 'Yugoslavia',
 nan}

In [ ]:
# merged.head(100)

###질문을 해결해보자 (데이터 뜯어보기)

#####이름

같은 ID인데 이름이 다른 경우가 있을까?

In [ ]:
id_name_counts = merged.groupby('ID')['Name'].nunique()
is_all_unique_name = (id_name_counts == 1).all()
is_all_unique_name

True

######이름 대신 참여연도수로 수치 적용하기

일단 참여한 것으로 메달 획들 가능성을 상승시켜줍니다.

In [ ]:
# merged['ID'] = (merged['Year'] / merged['Year'].max())

In [ ]:
# 'Year' 열을 'ID' 열 바로 뒤로 옮기기
cols = list(merged.columns)
# 'Year' 열을 제거
cols.remove('Year')
# 'ID' 인덱스 찾기
id_index = cols.index('ID')
# 'Year' 열을 'ID' 바로 뒤에 삽입
cols.insert(id_index + 1, 'Year')
# 열 순서 재조정
merged = merged[cols]

In [ ]:
merged.sort_values('Year', ascending=False)
merged.head()

# original_athlete.head()

# original_athlete['Year'].min()
# original_athlete['Year'].max()

# merged['ID'] = (merged['Year'] / (merged['Year'].max()))

merged['ID'] = (merged['Year'] - merged['Year'].min() + 1) / merged['Year'].max() * 10

In [ ]:
# 최대값을 가진 행들을 선택
# # max_year_rows
# merged[merged['Year'] == merged['Year'].max()]

merged[merged['Year'] == merged['Year'].min()]

,ID,Year,Name,Sex,Age,Height,Weight,Team,NOC,Games,Season,City,Sport,Event,Medal,region,notes
3079,0.00496,1896,Aristidis Akratopoulos,M,NaN,NaN,NaN,Greece,GRE,1896 Summer,Summer,Athina,Tennis,Tennis Men's Singles,NaN,Greece,NaN
3080,0.00496,1896,Aristidis Akratopoulos,M,NaN,NaN,NaN,Greece-3,GRE,1896 Summer,Summer,Athina,Tennis,Tennis Men's Doubles,NaN,Greece,NaN
3081,0.00496,1896,"Konstantinos ""Kostas"" Akratopoulos",M,NaN,NaN,NaN,Greece,GRE,1896 Summer,Summer,Athina,Tennis,Tennis Men's Singles,NaN,Greece,NaN
3082,0.00496,1896,"Konstantinos ""Kostas"" Akratopoulos",M,NaN,NaN,NaN,Greece-3,GRE,1896 Summer,Summer,Athina,Tennis,Tennis Men's Doubles,NaN,Greece,NaN
7348,0.00496,1896,Anastasios Andreou,M,NaN,NaN,NaN,Greece,GRE,1896 Summer,Summer,Athina,Athletics,Athletics Men's 110 metres Hurdles,NaN,Greece,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270473,0.00496,1896,Khristos Zoumis,M,NaN,NaN,NaN,Greece,GRE,1896 Summer,Summer,Athina,Athletics,Athletics Men's Triple Jump,NaN,Greece,NaN
270914,0.00496,1896,"Jules Alexis ""Louis"" Zutter",M,30.0,NaN,NaN,Switzerland,SUI,1896 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Horse Vault,Silver,Switzerland,NaN
270915,0.00496,1896,"Jules Alexis ""Louis"" Zutter",M,30.0,NaN,NaN,Switzerland,SUI,1896 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Parallel Bars,Silver,Switzerland,NaN
270916,0.00496,1896,"Jules Alexis ""Louis"" Zutter",M,30.0,NaN,NaN,Switzerland,SUI,1896 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Horizontal Bar,NaN,Switzerland,NaN


In [ ]:
merged = merged.sort_values(by='Year')

In [ ]:
merged.drop(columns=['Name'])

,ID,Year,Sex,Age,Height,Weight,Team,NOC,Games,Season,City,Sport,Event,Medal,region,notes
214333,0.004960,1896,M,19.0,NaN,NaN,Germany,GER,1896 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Pommelled Horse,NaN,Germany,NaN
244717,0.004960,1896,M,19.0,NaN,NaN,France,FRA,1896 Summer,Summer,Athina,Athletics,Athletics Men's Triple Jump,Silver,France,NaN
244716,0.004960,1896,M,19.0,NaN,NaN,France,FRA,1896 Summer,Summer,Athina,Athletics,Athletics Men's Long Jump,NaN,France,NaN
23912,0.004960,1896,M,25.0,NaN,NaN,Germany,GER,1896 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Horse Vault,NaN,Germany,NaN
23913,0.004960,1896,M,25.0,NaN,NaN,Germany,GER,1896 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Parallel Bars,NaN,Germany,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142355,0.600198,2016,M,37.0,166.0,60.0,Colombia,COL,2016 Summer,Summer,Rio de Janeiro,Athletics,Athletics Men's 20 kilometres Walk,NaN,Colombia,NaN
47729,0.600198,2016,M,20.0,152.0,53.0,Italy,ITA,2016 Summer,Summer,Rio de Janeiro,Rowing,Rowing Men's Coxed Eights,NaN,Italy,NaN
47728,0.600198,2016,F,23.0,173.0,71.0,Canada,CAN,2016 Summer,Summer,Rio de Janeiro,Football,Football Women's Football,Bronze,Canada,NaN
47746,0.600198,2016,M,21.0,194.0,85.0,Italy,ITA,2016 Summer,Summer,Rio de Janeiro,Swimming,Swimming Men's 200 metres Freestyle,NaN,Italy,NaN


#####메달

######메달을 3,2,1 점수로 처리해보자

In [ ]:
merged['MedalScore'] = merged['Medal'].map({'Gold': 3, 'Silver': 2, 'Bronze': 1, np.nan: 0})

In [ ]:
merged.head()

,ID,Year,Name,Sex,Age,Height,Weight,Team,NOC,Games,Season,City,Sport,Event,Medal,region,notes,MedalScore
214333,0.00496,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,Germany,GER,1896 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Pommelled Horse,NaN,Germany,NaN,0
244717,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,Summer,Athina,Athletics,Athletics Men's Triple Jump,Silver,France,NaN,2
244716,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,Summer,Athina,Athletics,Athletics Men's Long Jump,NaN,France,NaN,0
23912,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Horse Vault,NaN,Germany,NaN,0
23913,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,Summer,Athina,Gymnastics,Gymnastics Men's Parallel Bars,NaN,Germany,NaN,0


In [ ]:
for medal_name in set(merged['Medal']):
  if medal_name != 'NaN':
    merged[medal_name] = merged['Medal'].apply(lambda x: 1 if x == medal_name else 0)

In [ ]:
# merged = merged.dropna(axis=1)
merged.columns = [col if not pd.isna(col) else 'Unnamed' for col in merged.columns]


In [ ]:
merged.head()

,ID,Year,Name,Sex,Age,Height,Weight,Team,NOC,Games,...,Sport,Event,Medal,region,notes,MedalScore,Gold,Unnamed,Silver,Bronze
214333,0.00496,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,Germany,GER,1896 Summer,...,Gymnastics,Gymnastics Men's Pommelled Horse,NaN,Germany,NaN,0,0,0,0,0
244717,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,...,Athletics,Athletics Men's Triple Jump,Silver,France,NaN,2,0,0,1,0
244716,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,...,Athletics,Athletics Men's Long Jump,NaN,France,NaN,0,0,0,0,0
23912,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,...,Gymnastics,Gymnastics Men's Horse Vault,NaN,Germany,NaN,0,0,0,0,0
23913,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,...,Gymnastics,Gymnastics Men's Parallel Bars,NaN,Germany,NaN,0,0,0,0,0


In [ ]:
merged.drop(columns=['Unnamed'])

,ID,Year,Name,Sex,Age,Height,Weight,Team,NOC,Games,...,City,Sport,Event,Medal,region,notes,MedalScore,Gold,Silver,Bronze
214333,0.004960,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,Germany,GER,1896 Summer,...,Athina,Gymnastics,Gymnastics Men's Pommelled Horse,NaN,Germany,NaN,0,0,0,0
244717,0.004960,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,...,Athina,Athletics,Athletics Men's Triple Jump,Silver,France,NaN,2,0,1,0
244716,0.004960,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,...,Athina,Athletics,Athletics Men's Long Jump,NaN,France,NaN,0,0,0,0
23912,0.004960,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,...,Athina,Gymnastics,Gymnastics Men's Horse Vault,NaN,Germany,NaN,0,0,0,0
23913,0.004960,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,...,Athina,Gymnastics,Gymnastics Men's Parallel Bars,NaN,Germany,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142355,0.600198,2016,Luis Fernando Lpez Erazo,M,37.0,166.0,60.0,Colombia,COL,2016 Summer,...,Rio de Janeiro,Athletics,Athletics Men's 20 kilometres Walk,NaN,Colombia,NaN,0,0,0,0
47729,0.600198,2016,Enrico D'Aniello,M,20.0,152.0,53.0,Italy,ITA,2016 Summer,...,Rio de Janeiro,Rowing,Rowing Men's Coxed Eights,NaN,Italy,NaN,0,0,0,0
47728,0.600198,2016,Sabrina D'Angelo,F,23.0,173.0,71.0,Canada,CAN,2016 Summer,...,Rio de Janeiro,Football,Football Women's Football,Bronze,Canada,NaN,1,0,0,1
47746,0.600198,2016,Andrea Mitchell D'Arrigo,M,21.0,194.0,85.0,Italy,ITA,2016 Summer,...,Rio de Janeiro,Swimming,Swimming Men's 200 metres Freestyle,NaN,Italy,NaN,0,0,0,0


#####성별

In [ ]:
# def encoder(temp_list, column_name):
#   for item in temp_list:
#     merged[item] = merged[column_name].apply(lambda x: 1 if x == item else 0)

In [ ]:

# # 'Male'과 'Female' 열 추가
merged['Male'] = merged['Sex'].apply(lambda x: 1 if x == 'M' else 0)
merged['Female'] = merged['Sex'].apply(lambda x: 1 if x == 'F' else 0)

# #numpy 방식
# merged['Male'] = np.where(merged['Sex'] == 'M', 1, 0)
# merged['Female'] = np.where(merged['Sex'] == 'F', 1, 0)

In [ ]:
merged.head()

,ID,Year,Name,Sex,Age,Height,Weight,Team,NOC,Games,...,Medal,region,notes,MedalScore,Gold,Unnamed,Silver,Bronze,Male,Female
214333,0.00496,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,Germany,GER,1896 Summer,...,NaN,Germany,NaN,0,0,0,0,0,1,0
244717,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,...,Silver,France,NaN,2,0,0,1,0,1,0
244716,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,...,NaN,France,NaN,0,0,0,0,0,1,0
23912,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,...,NaN,Germany,NaN,0,0,0,0,0,1,0
23913,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,...,NaN,Germany,NaN,0,0,0,0,0,1,0


######스포츠 종목

In [ ]:
# 종목 원핫인코딩

for sport_name in set(merged['Sport']):
  merged[sport_name] = merged['Sport'].apply(lambda x: 1 if x == sport_name else 0)

In [ ]:
merged.head()

,ID,Year,Name,Sex,Age,Height,Weight,Team,NOC,Games,...,Taekwondo,Skeleton,Sailing,Boxing,Diving,Jeu De Paume,Lacrosse,Fencing,Badminton,Art Competitions
214333,0.00496,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,Germany,GER,1896 Summer,...,0,0,0,0,0,0,0,0,0,0
244717,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,...,0,0,0,0,0,0,0,0,0,0
244716,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,France,FRA,1896 Summer,...,0,0,0,0,0,0,0,0,0,0
23912,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,...,0,0,0,0,0,0,0,0,0,0
23913,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,Germany,GER,1896 Summer,...,0,0,0,0,0,0,0,0,0,0


#####신장, 체중

In [ ]:
# id_weight_counts = merged.groupby('ID')['Name'].nunique()
# is_all_unique_weight = (id_weight_counts == 1).all()
# is_all_unique_weight

######신장, 체중 BMI로 처리해보자

BMI를 미리 계산해두고
비교할 때 비교군별로 신장에 따른

In [ ]:
# BMI 계산 (BMI = 몸무게(kg) / (키(m)^2))
merged['BMI'] = merged['Weight'] / (merged['Height'])** 2 * 100

In [ ]:
# BMI 대비 신장 내림차순 정렬

In [ ]:
merged[merged['BMI'].isna() == False]

,ID,Year,Name,Sex,Age,Height,Weight,Team,NOC,Games,...,Skeleton,Sailing,Boxing,Diving,Jeu De Paume,Lacrosse,Fencing,Badminton,Art Competitions,BMI
75128,0.004960,1896,Carl Galle,M,23.0,154.0,45.0,Germany,GER,1896 Summer,...,0,0,0,0,0,0,0,0,0,0.189745
47375,0.004960,1896,"Thomas Pelham ""Tom"" Curtis",M,23.0,176.0,66.0,United States,USA,1896 Summer,...,0,0,0,0,0,0,0,0,0,0.213068
47374,0.004960,1896,"Thomas Pelham ""Tom"" Curtis",M,23.0,176.0,66.0,United States,USA,1896 Summer,...,0,0,0,0,0,0,0,0,0,0.213068
32334,0.004960,1896,"Thomas Edmund ""Tom"" Burke",M,21.0,183.0,66.0,United States,USA,1896 Summer,...,0,0,0,0,0,0,0,0,0,0.197080
32333,0.004960,1896,"Thomas Edmund ""Tom"" Burke",M,21.0,183.0,66.0,United States,USA,1896 Summer,...,0,0,0,0,0,0,0,0,0,0.197080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142355,0.600198,2016,Luis Fernando Lpez Erazo,M,37.0,166.0,60.0,Colombia,COL,2016 Summer,...,0,0,0,0,0,0,0,0,0,0.217738
47729,0.600198,2016,Enrico D'Aniello,M,20.0,152.0,53.0,Italy,ITA,2016 Summer,...,0,0,0,0,0,0,0,0,0,0.229398
47728,0.600198,2016,Sabrina D'Angelo,F,23.0,173.0,71.0,Canada,CAN,2016 Summer,...,0,0,0,0,0,0,0,0,0,0.237228
47746,0.600198,2016,Andrea Mitchell D'Arrigo,M,21.0,194.0,85.0,Italy,ITA,2016 Summer,...,0,0,0,0,0,0,0,0,0,0.225848


In [ ]:
cols = merged.columns.tolist()

# 'Weight' 컬럼의 인덱스를 찾아서
weight_index = cols.index('Weight')
# 'BMI' 컬럼을 'Weight' 컬럼 바로 뒤로 이동
# 'BMI' 컬럼을 리스트에서 제거하고, 'Weight' 컬럼 바로 뒤에 삽입
cols.insert(weight_index + 1, cols.pop(cols.index('BMI')))

# 데이터프레임을 새로운 컬럼 순서로 업데이트
merged = merged[cols]

In [ ]:
merged

,ID,Year,Name,Sex,Age,Height,Weight,BMI,Host Country,Team,...,Boxing,Diving,Jeu De Paume,Lacrosse,Fencing,Badminton,Art Competitions,Medal_Won,Host,Event Gender
0,0.004960,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,1,0,Men's
1,0.004960,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,0,0,1,0,Men's
2,0.004960,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,0,0,1,0,Men's
3,0.004960,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,1,0,Men's
4,0.004960,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,1,0,Men's
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455542,0.600198,2016,Sabrina D'Angelo,F,23.0,173.0,71.0,0.237228,Brazil,Canada,...,0,0,0,0,0,0,0,1,0,Women's
455543,0.600198,2016,Andrea Mitchell D'Arrigo,M,21.0,194.0,85.0,0.225848,Norway,Italy,...,0,0,0,0,0,0,0,1,0,Men's
455544,0.600198,2016,Andrea Mitchell D'Arrigo,M,21.0,194.0,85.0,0.225848,Brazil,Italy,...,0,0,0,0,0,0,0,1,0,Men's
455545,0.600198,2016,Blair Tarrant,M,26.0,185.0,83.0,0.242513,Norway,New Zealand,...,0,0,0,0,0,0,0,1,0,Men's


#####올림픽 개최국 베네핏 (컬럼 추가)

######이벤트별 개최지를 표시하자

In [ ]:
host.head()

,Type,GamesUrl,Disciplines,DisciplinesList,Country,Date,Athletes,Countries,Events,City,Year
0,summergames,https://www.olympic.org/athens-1896,10,"['Athletics', 'Cycling Road', 'Cycling Track',...",Greece,06 Apr - 15 Apr,241.0,14.0,43.0,Athens,1896
1,summergames,https://www.olympic.org/paris-1900,20,"['Archery', 'Athletics', 'Basque Pelota', 'Cri...",France,14 May - 28 Oct,997.0,24.0,95.0,Paris,1900
2,summergames,https://www.olympic.org/st-louis-1904,19,"['Archery', 'Athletics', 'Basketball', 'Boxing...",United States of America,01 Jul - 23 Nov,651.0,12.0,95.0,St Louis,1904
3,summergames,https://www.olympic.org/london-1908,25,"['Archery', 'Athletics', 'Boxing', 'Cycling Tr...",Great Britain,27 Apr - 31 Oct,2008.0,22.0,110.0,London,1908
4,summergames,https://www.olympic.org/stockholm-1912,18,"['Athletics', 'Cycling Road', 'Diving', 'Eques...",Sweden,05 May - 27 Jul,2407.0,28.0,102.0,Stockholm,1912


In [ ]:
host.rename(columns={'Country': 'Host Country'}, inplace=True)
host_country = host[['Year', 'Host Country']]
host_country.head()

,Year,Host Country
0,1896,Greece
1,1900,France
2,1904,United States of America
3,1908,Great Britain
4,1912,Sweden


In [ ]:
merged = pd.merge(merged, host_country, on='Year' , how='left')


In [ ]:
merged.head()

,ID,Year,Name,Sex,Age,Height,Weight,BMI,Team,NOC,...,Skeleton,Sailing,Boxing,Diving,Jeu De Paume,Lacrosse,Fencing,Badminton,Art Competitions,Host Country
0,0.00496,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,NaN,Germany,GER,...,0,0,0,0,0,0,0,0,0,Greece
1,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,France,FRA,...,0,0,0,0,0,0,0,0,0,Greece
2,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,France,FRA,...,0,0,0,0,0,0,0,0,0,Greece
3,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Germany,GER,...,0,0,0,0,0,0,0,0,0,Greece
4,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Germany,GER,...,0,0,0,0,0,0,0,0,0,Greece


In [ ]:
cols = merged.columns.tolist()

bmi_index = cols.index('BMI')
cols.insert(bmi_index + 1, cols.pop(cols.index('Host Country')))

# 데이터프레임을 새로운 컬럼 순서로 업데이트합니다.
merged = merged[cols]

In [ ]:
merged

,ID,Year,Name,Sex,Age,Height,Weight,BMI,Host Country,Team,...,Boxing,Diving,Jeu De Paume,Lacrosse,Fencing,Badminton,Art Competitions,Medal_Won,Host,Event Gender
0,0.004960,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,1,0,Men's
1,0.004960,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,0,0,1,0,Men's
2,0.004960,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,0,0,1,0,Men's
3,0.004960,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,1,0,Men's
4,0.004960,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,1,0,Men's
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455542,0.600198,2016,Sabrina D'Angelo,F,23.0,173.0,71.0,0.237228,Brazil,Canada,...,0,0,0,0,0,0,0,1,0,Women's
455543,0.600198,2016,Andrea Mitchell D'Arrigo,M,21.0,194.0,85.0,0.225848,Norway,Italy,...,0,0,0,0,0,0,0,1,0,Men's
455544,0.600198,2016,Andrea Mitchell D'Arrigo,M,21.0,194.0,85.0,0.225848,Brazil,Italy,...,0,0,0,0,0,0,0,1,0,Men's
455545,0.600198,2016,Blair Tarrant,M,26.0,185.0,83.0,0.242513,Norway,New Zealand,...,0,0,0,0,0,0,0,1,0,Men's


In [ ]:
merged['Medal_Won'] = merged['MedalScore'].apply(lambda x: 1 if not 0 else 0)

In [ ]:
# 'Host' 열을 생성하되, 'Host Country'와 'Region'이 같으면 1, 다르면 0을 할당합니다.
merged['Host'] = (merged['Host Country'] == merged['region']).astype(int)

In [ ]:
merged.head()

,ID,Year,Name,Sex,Age,Height,Weight,BMI,Host Country,Team,...,Sailing,Boxing,Diving,Jeu De Paume,Lacrosse,Fencing,Badminton,Art Competitions,Medal_Won,Host
0,0.00496,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,0,1,0
1,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,0,0,0,1,0
2,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,0,0,0,1,0
3,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,0,1,0
4,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,0,1,0


패널티를 메달획득률에 곱한 Medal unbiased 컬럼을 추가하자

패널티 수치 (메달에 대한 베네핏) 컬럼을 추가하자

In [ ]:
countries = list(set(merged['region']))

# 국가별로 데이터 그룹화
benefit_as_host = {}
for a_region in countries:

    # 개최국일 때의 메달 획득률
    host_medal_rate = merged[merged['Host'] == 1]['MedalScore'].mean()

    # 비개최국일 때의 메달 획득률
    non_host_medal_rate = merged[merged['Host'] == 0]['MedalScore'].mean()

    # 메달 획득률의 비율 계산 (0으로 나누는 것을 방지)
    if non_host_medal_rate > 0:
        benefit_as_host[a_region] = host_medal_rate / non_host_medal_rate
    else:
        benefit_as_host[a_region] = None  # 비개최국에서 메달을 하나도 획득하지 못한 경우

    merged['Penalty'] = non_host_medal_rate / host_medal_rate

    merged['Medal unbiased'] = merged['MedalScore'] * merged['Penalty']

In [ ]:
benefit_as_host

benefit_df  = pd.DataFrame(benefit_as_host.items(), columns=['Country', 'Benefit'])



In [ ]:
benefit_df['Penalty'] = benefit_df['Country'].map(benefit_as_host)

In [ ]:
benefit_df

,Country,Benefit,Penalty
0,Chile,1.239639,1.239639
1,Sweden,1.239639,1.239639
2,"Virgin Islands, US",1.239639,1.239639
3,Belize,1.239639,1.239639
4,North Korea,1.239639,1.239639
...,...,...,...
201,Honduras,1.239639,1.239639
202,Puerto Rico,1.239639,1.239639
203,Central African Republic,1.239639,1.239639
204,Australia,1.239639,1.239639


In [ ]:
# host_medal_rate

In [ ]:
# non_host_medal_rate

######올림픽 개최국 베네핏을 메달 획득에 패널티로 적용해보자

In [ ]:
merged['Benefit'] = 1 / merged['Penalty']

In [ ]:
merged.head()

,ID,Year,Name,Sex,Age,Height,Weight,BMI,Host Country,Team,...,Jeu De Paume,Lacrosse,Fencing,Badminton,Art Competitions,Medal_Won,Host,Penalty,Medal unbiased,Benefit
0,0.00496,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,1,0,0.806687,0.000000,1.239639
1,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,1,0,0.806687,1.613374,1.239639
2,0.00496,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,1,0,0.806687,0.000000,1.239639
3,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,1,0,0.806687,0.000000,1.239639
4,0.00496,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,1,0,0.806687,0.000000,1.239639


In [ ]:
# Event 컬럼의 위치(index) 가져오기
event_idx = merged.columns.get_loc('Host Country')

# Participants 컬럼을 Event 컬럼 다음 위치로 옮기기
cols = list(merged.columns)
cols.insert(event_idx + 1, cols.pop(cols.index('Host')))

# 새로운 순서로 데이터프레임 재구성
merged = merged[cols]

In [ ]:
# def relocate_col(df, near_col_name, new_col_name):
#   col_idx = df.columns.get_loc(near_col_name)

#   # Participants 컬럼을 Event 컬럼 다음 위치로 옮기기
#   cols = list(df.columns)
#   cols.insert(col_idx + 1, cols.pop(cols.index(new_col_name)))

#   # 새로운 순서로 데이터프레임 재구성
#   df = df[cols]

# def relocate_col(df, near_col_name, new_col_name):
#   # Event 컬럼의 위치(index) 가져오기
#   event_idx = df.columns.get_loc(near_col_name)

#   # Participants 컬럼을 Event 컬럼 다음 위치로 옮기기
#   cols = list(df.columns)
#   cols.insert(event_idx + 1, cols.pop(cols.index(new_col_name)))

#   # 새로운 순서로 데이터프레임 재구성
#   df = df[cols]
#   return df

In [ ]:
# relocate_col(merged, 'Host', 'Penalty')

event_idx = merged.columns.get_loc('Host')

cols = list(merged.columns)
cols.insert(event_idx + 1, cols.pop(cols.index('Penalty')))

merged = merged[cols]

In [ ]:
# relocate_col(merged, 'Penalty', 'Benefit')

event_idx = merged.columns.get_loc('Penalty')

cols = list(merged.columns)
cols.insert(event_idx + 1, cols.pop(cols.index('Benefit')))

merged = merged[cols]

In [ ]:
merged

,ID,Year,Name,Sex,Age,Height,Weight,BMI,Host Country,Team,...,Sailing,Boxing,Diving,Jeu De Paume,Lacrosse,Fencing,Badminton,Art Competitions,Medal_Won,Host
0,0.004960,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,0,1,0
1,0.004960,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,0,0,0,1,0
2,0.004960,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,0,0,0,1,0
3,0.004960,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,0,1,0
4,0.004960,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455542,0.600198,2016,Sabrina D'Angelo,F,23.0,173.0,71.0,0.237228,Brazil,Canada,...,0,0,0,0,0,0,0,0,1,0
455543,0.600198,2016,Andrea Mitchell D'Arrigo,M,21.0,194.0,85.0,0.225848,Norway,Italy,...,0,0,0,0,0,0,0,0,1,0
455544,0.600198,2016,Andrea Mitchell D'Arrigo,M,21.0,194.0,85.0,0.225848,Brazil,Italy,...,0,0,0,0,0,0,0,0,1,0
455545,0.600198,2016,Blair Tarrant,M,26.0,185.0,83.0,0.242513,Norway,New Zealand,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# 개최국 혜택 있는 국가는 Penalty 부여 (비율), 개최한 적 없는 국가는 1로 처리
# 신체정보 적절성 파악하기 위한 모델은 위와 같은 조건.
# 특성 중요도 파악 시에는 베네핏은 베네핏으로. 또는 베네핏은 주지 않는다. (트리모델의 결과값이 곧 베네핏)

#####이벤트명

일반적으로는:
* 종목
* 참여자 성별
* 참여자 인원수

토큰화 후 알아보자.

######사전에 생성된 함수

함수 생성 (일치하는 데이터가 몇 개나 있는지 조사하는 함수) - 사용하지 않음

In [ ]:
# def inspect_column_by_list(df, column_name, reference_list):
#     # 특정 열(column_name)에서 각 행의 문자열을 가져와서, 특정 리스트에 있는 문자열과 일치하는지를 확인합니다.
#     matched_rows = merged[merged[column_name].apply(lambda x: any(substring in x for substring in reference_list))]

#     return matched_rows

In [ ]:
# num_list = ['Pairs', 'Two', 'Fours', 'Four', 'Eights']

# eventwith_num = inspect_column_by_list(merged, 'Event', num_list)

함수 생성 (일치하지 않아 남은 문자열을 저장할 함수) - 사용하지 않음

In [ ]:
# # 데이터프레임의 특정 열에서 특정 리스트 기준으로 일치하지 않는 문자열을 새로운 리스트에 저장하는 함수 정의
# def exclude_matching_strings(df, column_name, reference_list):
#     # 일치하지 않는 문자열을 저장할 빈 리스트 생성
#     excluded_strings = []

#     # 데이터프레임의 특정 열에서 각 행의 문자열을 가져와서, 특정 리스트에 포함된 문자열과 일치하지 않는 경우에만 추가합니다.
#     for row in df[column_name]:
#         if not any(substring in row for substring in reference_list):
#             excluded_strings.append(row)

#     return excluded_strings

함수생성 (이벤트 컬럼에서 문자열을 제거하며 조사할 함수)

In [ ]:
def extract_remaining_strings(df, column_name, reference_list):
    remaining_strings = []

    for event in df[column_name]:
        for category in reference_list:
            if category in event:
                event = event.replace(category, '').strip()
        remaining_strings.append(event)

    return remaining_strings

######경기별1. 성별

In [ ]:
gender_categories = ["Men's", "Women's", "Mixed"]

# # 함수를 사용하여 특정 열에서 특정 리스트 기준으로 일치하지 않는 문자열을 새로운 리스트에 저장합니다.
# excluded_strings = exclude_matching_strings(merged, 'Event', event_name)
# excluded_strings


# 함수를 사용하여 특정 열에서 특정 리스트 기준으로 일치하는 문자열에서 나머지 문자열을 추출하여 새로운 리스트에 저장합니다.
remaining_strings = extract_remaining_strings(merged, 'Event', gender_categories)

In [ ]:
remaining_strings

['Gymnastics  Pommelled Horse',
 'Athletics  Triple Jump',
 'Athletics  Long Jump',
 'Gymnastics  Horse Vault',
 'Gymnastics  Parallel Bars',
 'Gymnastics  Parallel Bars, Teams',
 'Gymnastics  Horizontal Bar',
 'Gymnastics  Horizontal Bar, Teams',
 'Gymnastics  Rings',
 'Gymnastics  Pommelled Horse',
 'Athletics  1,500 metres',
 'Shooting  Military Rifle, 200 metres',
 'Athletics  Marathon',
 'Athletics  800 metres',
 'Tennis  Doubles',
 'Tennis  Singles',
 'Fencing  Foil, Individual',
 'Swimming  1,200 metres Freestyle',
 'Swimming  500 metres Freestyle',
 'Athletics  Discus Throw',
 'Athletics  Long Jump',
 'Athletics  Marathon',
 'Cycling  100 kilometres',
 'Cycling  333 metres Time Trial',
 'Gymnastics  Horse Vault',
 'Gymnastics  Parallel Bars',
 'Shooting  Military Pistol, 25 metres',
 'Tennis  Doubles',
 'Shooting  Muzzle-Loading Pistol, 25 metres',
 'Shooting  Military Rifle, 200 metres',
 'Gymnastics  Parallel Bars, Teams',
 'Gymnastics  Horizontal Bar',
 'Gymnastics  Horizont

토큰화 함수 생성 (리스트를 참조해 일치하는 부분 새로운 리스트로 저장하는 함수)

In [ ]:
# # 토큰화 함수 정의
# def tokenize(input_str):

#   # 두 개의 리스트를 저장할 빈 리스트
#   gender_token = []
#   sport_token = []
#   none_token = [] # 아무 집합에도 속하지 않는 경우를 저장할 리스트 추가

#   # 입력 토큰들을 iterate하면서 집합에 속하는지 확인하여 분류
#   for token in input_str:
#       found = False  # 토큰이 어떤 집합에도 속하지 않았는지 여부를 확인하기 위한 변수

#       # token을 set1의 요소들로부터 일치하는 부분을 찾아서 token1에 저장
#       for item in gender_categories:
#           if item in token:
#               gender_token.append(item)
#               found = True  # set1에 속하는 경우 found를 True로 설정
#               break  # 일치하는 것을 찾았으면 루프 탈출

#       # token을 set2의 요소들로부터 일치하는 부분을 찾아서 token2에 저장
#       for item in sport_name:
#           if item in token:
#               sport_token.append(item)
#               found = True  # set2에 속하는 경우 found를 True로 설정
#               break  # 일치하는 것을 찾았으면 루프 탈출

#       # 토큰이 어느 집합에도 속하지 않는 경우
#       if not found:
#           none_token.append(token)

#   # 분류된 토큰 리스트를 반환
#   return gender_token, sport_token, none_token

In [ ]:
# 토큰화 함수 정의
def tokenize(input_list, ref_list):

  token = []

  # 입력 토큰들을 iterate하면서 집합에 속하는지 확인하여 분류
  for item in ref_list:
    if item in input_list:
      token.append(item)
  return token

In [ ]:
# Column1에 tokenize 함수 적용하여 list로 저장
event_gender = merged['Event'].apply(lambda x: tokenize(x, gender_categories))

In [ ]:
event_gender

0           [Men's]
1           [Men's]
2           [Men's]
3           [Men's]
4           [Men's]
            ...    
455542    [Women's]
455543      [Men's]
455544      [Men's]
455545      [Men's]
455546      [Men's]
Name: Event, Length: 455547, dtype: object

저장된 리스트로 컬럼 추가

In [ ]:
# 'Event' 컬럼에서 gender_categories의 요소를 찾고, 있으면 해당 요소를 'Event Gender'에 저장, 없으면 NaN 저장
def find_category(event):
    for category in gender_categories:
        if category in event:
            return category
    return np.nan

merged['Event Gender'] = merged['Event'].apply(find_category)

In [ ]:
merged

,ID,Year,Name,Sex,Age,Height,Weight,BMI,Host Country,Team,...,Boxing,Diving,Jeu De Paume,Lacrosse,Fencing,Badminton,Art Competitions,Medal_Won,Host,Event Gender
0,0.004960,1896,Fritz Richard Gustav Schuft,M,19.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,1,0,Men's
1,0.004960,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,0,0,1,0,Men's
2,0.004960,1896,Pierre Alexandre Tuffri,M,19.0,NaN,NaN,NaN,Greece,France,...,0,0,0,0,0,0,0,1,0,Men's
3,0.004960,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,1,0,Men's
4,0.004960,1896,Conrad Helmut Fritz Bcker,M,25.0,NaN,NaN,NaN,Greece,Germany,...,0,0,0,0,0,0,0,1,0,Men's
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455542,0.600198,2016,Sabrina D'Angelo,F,23.0,173.0,71.0,0.237228,Brazil,Canada,...,0,0,0,0,0,0,0,1,0,Women's
455543,0.600198,2016,Andrea Mitchell D'Arrigo,M,21.0,194.0,85.0,0.225848,Norway,Italy,...,0,0,0,0,0,0,0,1,0,Men's
455544,0.600198,2016,Andrea Mitchell D'Arrigo,M,21.0,194.0,85.0,0.225848,Brazil,Italy,...,0,0,0,0,0,0,0,1,0,Men's
455545,0.600198,2016,Blair Tarrant,M,26.0,185.0,83.0,0.242513,Norway,New Zealand,...,0,0,0,0,0,0,0,1,0,Men's


In [ ]:
# merged['Event_Male'] = merged['Event Gender'].apply(lambda x: 1 if x == "[Men's]" else 0)
# merged['Event_Female'] = merged['Event Gender'].apply(lambda x: 1 if x == "[Women's]" else 0)

######경기별2. 참여인원수

In [ ]:
num_list = ['Individual', 'Pairs', 'Two', 'Doubles', 'Duet', 'Three Person', 'Fours', 'Four', 'Quadruple', 'Quadruples', 'Eights', 'Team', 'Relay']
num_dict = {'Individual': 1, 'Singles': 1, 'One': 1, 'Pairs': 2, 'Two': 2, 'Doubles': 2, 'Duet': 2, 'Three Person': 3, 'Fours': 4, 'Four': 4, 'Quadruple': 4, 'Quadruples': 4, 'Eights': 8, 'Team': 10, 'Relay': 10, 'Basketball': 10 ,'Football': 10 ,'Handball': 10 ,'Hockey': 10 ,'Rugby Sevens': 10 ,'Volleyball': 10 ,'Water Polo' : 10}

올림픽에서 9명 이상인 팀으로 출전해야 하는 종목
* Basketball
* Football
* Handball
* Hockey
* Rugby Sevens
* Volleyball
* Water Polo

In [ ]:
num_dict

{'Individual': 1,
 'Singles': 1,
 'Pairs': 2,
 'Two': 2,
 'Doubles': 2,
 'Duet': 2,
 'Three Person': 3,
 'Fours': 4,
 'Four': 4,
 'Quadruple': 4,
 'Quadruples': 4,
 'Eights': 8,
 'Team': 10,
 'Relay': 10,
 'Basketball': 10,
 'Football': 10,
 'Handball': 10,
 'Hockey': 10,
 'Rugby Sevens': 10,
 'Volleyball': 10,
 'Water Polo': 10}

In [ ]:
# Event 열을 조사하고, 딕셔너리의 값을 가져오거나 'NaN'을 할당하는 함수 정의
def get_value_from_dict(event):
    for key in num_dict:
        if key in event:
            return num_dict[key]
    return np.nan  # 딕셔너리에서 매칭되는 값이 없는 경우 NaN 반환

In [ ]:
# apply 함수를 사용하여 각 행에 함수 적용하여 결과 리스트 생성
participants = merged['Event'].apply(lambda x: get_value_from_dict(x))

In [ ]:
participants

0         10.0
1          NaN
2         10.0
3          NaN
4          NaN
          ... 
271111     2.0
271112     1.0
271113    10.0
271114     4.0
271115     4.0
Name: Event, Length: 271116, dtype: float64

In [ ]:
merged['Participants'] = participants
merged.head(100)

<ipython-input-112-095b175e7c60>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['Participants'] = participants


,MedalScore,Medal unbiased,Benefit,Penalty,Event,Sport,Sex,Host,Medal_Won,Age,...,region,Season,Year,Name,NOC,Games,Team,City,Event Gender,Participants
0,0,0.000000,1.239639,0.806687,Gymnastics Men's Pommelled Horse,Gymnastics,M,0,1,19.0,...,Germany,Summer,1896,Fritz Richard Gustav Schuft,GER,1896 Summer,Germany,Athina,[Men's],NaN
1,2,1.613374,1.239639,0.806687,Athletics Men's Triple Jump,Athletics,M,0,1,19.0,...,France,Summer,1896,Pierre Alexandre Tuffri,FRA,1896 Summer,France,Athina,[Men's],NaN
2,0,0.000000,1.239639,0.806687,Athletics Men's Long Jump,Athletics,M,0,1,19.0,...,France,Summer,1896,Pierre Alexandre Tuffri,FRA,1896 Summer,France,Athina,[Men's],NaN
3,0,0.000000,1.239639,0.806687,Gymnastics Men's Horse Vault,Gymnastics,M,0,1,25.0,...,Germany,Summer,1896,Conrad Helmut Fritz Bcker,GER,1896 Summer,Germany,Athina,[Men's],NaN
4,0,0.000000,1.239639,0.806687,Gymnastics Men's Parallel Bars,Gymnastics,M,0,1,25.0,...,Germany,Summer,1896,Conrad Helmut Fritz Bcker,GER,1896 Summer,Germany,Athina,[Men's],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0.000000,1.239639,0.806687,"Cycling Men's Road Race, Individual",Cycling,M,1,1,NaN,...,Greece,Summer,1896,Miltiadis Iatrou,GRE,1896 Summer,Greece,Athina,[Men's],1.0
96,0,0.000000,1.239639,0.806687,"Fencing Men's Sabre, Individual",Fencing,M,1,1,NaN,...,Greece,Summer,1896,Georgios Iatridis,GRE,1896 Summer,Greece,Athina,[Men's],1.0
97,0,0.000000,1.239639,0.806687,Gymnastics Men's Horse Vault,Gymnastics,M,0,1,19.0,...,Germany,Summer,1896,Georg Otto Hilmar,GER,1896 Summer,Germany,Athina,[Men's],NaN
98,0,0.000000,1.239639,0.806687,Gymnastics Men's Parallel Bars,Gymnastics,M,0,1,19.0,...,Germany,Summer,1896,Georg Otto Hilmar,GER,1896 Summer,Germany,Athina,[Men's],NaN


In [ ]:
pd.isnull(participants).sum()

151699

In [ ]:
# Event 컬럼의 위치(index) 가져오기
event_idx = merged.columns.get_loc('Event')

# Participants 컬럼을 Event 컬럼 다음 위치로 옮기기
cols = list(merged.columns)
cols.insert(event_idx + 1, cols.pop(cols.index('Participants')))

# 새로운 순서로 데이터프레임 재구성
merged = merged[cols]

In [ ]:
nan_participants.shape # 참여 인원수를 알 수 없는 항목들

(151699, 20)

In [ ]:
nan_participants = merged[merged['Participants'].isna()]
nan_participants

,MedalScore,Medal unbiased,Benefit,Penalty,Event,Sport,Sex,Host,Medal_Won,Age,...,NOC,Games,Team,City,Event Gender,Participants,Male,Female,Event_Male,Event_Female
0,0,0.000000,1.239639,0.806687,Gymnastics Men's Pommelled Horse,Gymnastics,M,0,1,19.0,...,GER,1896 Summer,Germany,Athina,[Men's],NaN,1,0,0,0
1,2,1.613374,1.239639,0.806687,Athletics Men's Triple Jump,Athletics,M,0,1,19.0,...,FRA,1896 Summer,France,Athina,[Men's],NaN,1,0,0,0
2,0,0.000000,1.239639,0.806687,Athletics Men's Long Jump,Athletics,M,0,1,19.0,...,FRA,1896 Summer,France,Athina,[Men's],NaN,1,0,0,0
3,0,0.000000,1.239639,0.806687,Gymnastics Men's Horse Vault,Gymnastics,M,0,1,25.0,...,GER,1896 Summer,Germany,Athina,[Men's],NaN,1,0,0,0
4,0,0.000000,1.239639,0.806687,Gymnastics Men's Parallel Bars,Gymnastics,M,0,1,25.0,...,GER,1896 Summer,Germany,Athina,[Men's],NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455528,0,0.000000,1.239639,0.806687,Gymnastics Men's Horizontal Bar,Gymnastics,M,0,1,27.0,...,IRL,2016 Summer,Ireland,Rio de Janeiro,[Men's],NaN,1,0,0,0
455537,0,0.000000,1.239639,0.806687,Athletics Men's 20 kilometres Walk,Athletics,M,0,1,37.0,...,COL,2016 Summer,Colombia,Rio de Janeiro,[Men's],NaN,1,0,0,0
455538,0,0.000000,1.239639,0.806687,Athletics Men's 20 kilometres Walk,Athletics,M,0,1,37.0,...,COL,2016 Summer,Colombia,Rio de Janeiro,[Men's],NaN,1,0,0,0
455543,0,0.000000,1.239639,0.806687,Swimming Men's 200 metres Freestyle,Swimming,M,0,1,21.0,...,ITA,2016 Summer,Italy,Rio de Janeiro,[Men's],NaN,1,0,0,0


In [ ]:
len(set(nan_participants['Event']))

533

In [ ]:
set(nan_participants['Event'])

{'Aeronautics Mixed Aeronautics',
 "Alpine Skiing Men's Combined",
 "Alpine Skiing Men's Downhill",
 "Alpine Skiing Men's Giant Slalom",
 "Alpine Skiing Men's Slalom",
 "Alpine Skiing Men's Super G",
 "Alpine Skiing Women's Combined",
 "Alpine Skiing Women's Downhill",
 "Alpine Skiing Women's Giant Slalom",
 "Alpine Skiing Women's Slalom",
 "Alpine Skiing Women's Super G",
 'Alpinism Mixed Alpinism',
 "Archery Men's Au Chapelet, 33 metres",
 "Archery Men's Au Chapelet, 50 metres",
 "Archery Men's Au Cordon Dore, 33 metres",
 "Archery Men's Au Cordon Dore, 50 metres",
 "Archery Men's Championnat Du Monde",
 "Archery Men's Continental Style",
 "Archery Men's Double American Round",
 "Archery Men's Double York Round",
 "Archery Men's Sur La Perche a La Herse",
 "Archery Men's Sur La Perche a La Pyramide",
 "Archery Men's Unknown Event",
 "Archery Women's Double Columbia Round",
 "Archery Women's Double National Round",
 'Art Competitions Mixed Architecture',
 'Art Competitions Mixed Archi

In [ ]:
# 일치하는 문자열을 저장할 리스트 생성
participants_spoken = []

# Event 열을 조사하여 일치하는 문자열 추출하여 리스트에 저장
# participants_spoken = merged['Event'].apply(lambda x: matching_strings.append(x) if any(word in x for word in num_dict) else None)

# Event 열을 조사하여 일치하는 값을 추출하여 리스트에 저장
for event in merged['Event']:
    found_value = NaN
    for key, value in num_dict.items():
        if key in event:
            found_value = value
            break  # 일치하는 첫 번째 값만 찾고 루프 종료
    matching_values.append(found_value if found_value is not None else np.nan)

participants_spoken

NameError: name 'NaN' is not defined

In [ ]:
matching_strings

["Cross Country Skiing Men's 4 x 10 kilometres Relay",
 "Cross Country Skiing Men's 4 x 10 kilometres Relay",
 "Cross Country Skiing Men's 4 x 10 kilometres Relay",
 "Cross Country Skiing Men's 4 x 10 kilometres Relay",
 "Athletics Women's 4 x 100 metres Relay",
 "Gymnastics Men's Individual All-Around",
 "Gymnastics Men's Team All-Around",
 "Gymnastics Men's Individual All-Around",
 "Gymnastics Men's Team All-Around",
 "Gymnastics Men's Team All-Around, Free System",
 'Sailing Mixed Three Person Keelboat',
 'Sailing Mixed Three Person Keelboat',
 "Sailing Men's Two Person Dinghy",
 "Sailing Men's Two Person Dinghy",
 "Bobsleigh Men's Four",
 "Swimming Men's 4 x 100 metres Medley Relay",
 "Gymnastics Men's Team All-Around, Free System",
 "Fencing Men's Foil, Individual",
 "Fencing Men's epee, Individual",
 "Fencing Men's epee, Team",
 'Equestrianism Mixed Jumping, Individual',
 "Cross Country Skiing Men's 4 x 10 kilometres Relay",
 "Athletics Men's 4 x 100 metres Relay",
 "Gymnastics M

In [ ]:
type(matching_strings)

list

In [ ]:
merged['Participants'] = participants

In [ ]:
merged.head(300)

,MedalScore,Medal unbiased,Benefit,Penalty,Event,Sport,Sex,Host,Medal_Won,Age,...,Year,Name,NOC,Games,Team,City,Event Gender,Participants,Male,Female
0,0,0.000000,1.239639,0.806687,Gymnastics Men's Pommelled Horse,Gymnastics,M,0,1,19.0,...,1896,Fritz Richard Gustav Schuft,GER,1896 Summer,Germany,Athina,[Men's],NaN,1,0
1,2,1.613374,1.239639,0.806687,Athletics Men's Triple Jump,Athletics,M,0,1,19.0,...,1896,Pierre Alexandre Tuffri,FRA,1896 Summer,France,Athina,[Men's],NaN,1,0
2,0,0.000000,1.239639,0.806687,Athletics Men's Long Jump,Athletics,M,0,1,19.0,...,1896,Pierre Alexandre Tuffri,FRA,1896 Summer,France,Athina,[Men's],NaN,1,0
3,0,0.000000,1.239639,0.806687,Gymnastics Men's Horse Vault,Gymnastics,M,0,1,25.0,...,1896,Conrad Helmut Fritz Bcker,GER,1896 Summer,Germany,Athina,[Men's],NaN,1,0
4,0,0.000000,1.239639,0.806687,Gymnastics Men's Parallel Bars,Gymnastics,M,0,1,25.0,...,1896,Conrad Helmut Fritz Bcker,GER,1896 Summer,Germany,Athina,[Men's],NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0,0.000000,1.239639,0.806687,Athletics Men's 800 metres,Athletics,M,1,1,18.0,...,1896,"Dimitrios Mikhail ""Dimitri"" Tombrof",GRE,1896 Summer,Greece,Athina,[Men's],NaN,1,0
296,3,2.420060,1.239639,0.806687,"Gymnastics Men's Parallel Bars, Teams",Gymnastics,M,0,1,19.0,...,1896,Fritz Richard Gustav Schuft,GER,1896 Summer,Germany,Athina,[Men's],10.0,1,0
297,3,2.420060,1.239639,0.806687,"Gymnastics Men's Horizontal Bar, Teams",Gymnastics,M,0,1,19.0,...,1896,Fritz Richard Gustav Schuft,GER,1896 Summer,Germany,Athina,[Men's],10.0,1,0
298,0,0.000000,1.239639,0.806687,"Shooting Men's Free Rifle, Three Positions, 30...",Shooting,M,1,1,NaN,...,1896,Georgios Diamantis,GRE,1896 Summer,Greece,Athina,[Men's],NaN,1,0


######경기별3. 거리

#######종목명 처리 (가독성을 위해 제거)

In [ ]:
# sport_name = set(merged['Sport'])
# # sport_name

# total_list = sport_name.union(set(gender_categories))

# remaining_strings = extract_remaining_strings(merged, 'Event', total_list)

In [ ]:
remaining_strings

['',
 'Extra-Lightweight',
 '',
 '',
 '500 metres',
 '1,000 metres',
 '500 metres',
 '1,000 metres',
 '500 metres',
 '1,000 metres',
 '10 kilometres',
 '50 kilometres',
 '10/15 kilometres Pursuit',
 '4 x 10 kilometres Relay',
 '10 kilometres',
 '30 kilometres',
 '10/15 kilometres Pursuit',
 '4 x 10 kilometres Relay',
 '10 kilometres',
 '50 kilometres',
 '10/15 kilometres Pursuit',
 '4 x 10 kilometres Relay',
 '10 kilometres',
 '30 kilometres',
 '10/15 kilometres Pursuit',
 '4 x 10 kilometres Relay',
 '100 metres',
 '4 x 100 metres Relay',
 '',
 '400 metres Freestyle',
 '30 kilometres',
 'Singles',
 'Windsurfer',
 'Windsurfer',
 '7.5 kilometres Sprint',
 '200 metres Breaststroke',
 '400 metres Breaststroke',
 '200 metres Breaststroke',
 '400 metres Breaststroke',
 '200 metres Breaststroke',
 '',
 'Individual All-Around',
 'Team All-Around',
 'Floor Exercise',
 'Horse Vault',
 'Parallel Bars',
 'Horizontal Bar',
 'Rings',
 'Pommelled Horse',
 'Individual All-Around',
 'Team All-Around',


######경기별4. 무게


####기타 시도

In [ ]:
merged['Team'].isnull().sum()

#####포괄 연도수

In [ ]:
merged.sort_values('Year', ascending=True)

* 1816 ~ 2016

#####regions 데이터프레임

* regions merge 가능 여부 확인
  * NOC 기준으로 국가 더해주기 (판단)
* notes 내용 확인

In [ ]:
print(regions.shape)
regions.head()

* NOC 코드에 맞춰서 나라 입력해주기


#####컬럼 정보 확인

notes 내용 확인

In [ ]:
regions[regions['notes'].notnull()]

* notes 정보 어떻게 활용할지 고민해보기

###랜덤포레스트를 위한 데이터

###신경망을 위한 데이터

In [ ]:
merged.head()

,MedalScore,Medal unbiased,Benefit,Penalty,Event,Sport,Sex,Host,Medal_Won,Age,...,region,Season,Year,Name,NOC,Games,Team,City,Event Gender,Participants
0,0,0.000000,1.239639,0.806687,Gymnastics Men's Pommelled Horse,Gymnastics,M,0,1,19.0,...,Germany,Summer,1896,Fritz Richard Gustav Schuft,GER,1896 Summer,Germany,Athina,[Men's],NaN
1,2,1.613374,1.239639,0.806687,Athletics Men's Triple Jump,Athletics,M,0,1,19.0,...,France,Summer,1896,Pierre Alexandre Tuffri,FRA,1896 Summer,France,Athina,[Men's],NaN
2,0,0.000000,1.239639,0.806687,Athletics Men's Long Jump,Athletics,M,0,1,19.0,...,France,Summer,1896,Pierre Alexandre Tuffri,FRA,1896 Summer,France,Athina,[Men's],NaN
3,0,0.000000,1.239639,0.806687,Gymnastics Men's Horse Vault,Gymnastics,M,0,1,25.0,...,Germany,Summer,1896,Conrad Helmut Fritz Bcker,GER,1896 Summer,Germany,Athina,[Men's],NaN
4,0,0.000000,1.239639,0.806687,Gymnastics Men's Parallel Bars,Gymnastics,M,0,1,25.0,...,Germany,Summer,1896,Conrad Helmut Fritz Bcker,GER,1896 Summer,Germany,Athina,[Men's],NaN


In [ ]:
for_dnn = merged[:]

In [ ]:
dnn_dropped = for_dnn.drop(['Sex', 'Year', 'Name', 'NOC', 'Games', 'Team', 'City', 'Season'], axis=1)

In [ ]:
dnn_dropped.head()

,MedalScore,Medal unbiased,Benefit,Penalty,Event,Sport,Host,Medal_Won,Age,ID,Host Country,region,Event Gender,Participants
0,0,0.000000,1.239639,0.806687,Gymnastics Men's Pommelled Horse,Gymnastics,0,1,19.0,0.00496,Greece,Germany,[Men's],NaN
1,2,1.613374,1.239639,0.806687,Athletics Men's Triple Jump,Athletics,0,1,19.0,0.00496,Greece,France,[Men's],NaN
2,0,0.000000,1.239639,0.806687,Athletics Men's Long Jump,Athletics,0,1,19.0,0.00496,Greece,France,[Men's],NaN
3,0,0.000000,1.239639,0.806687,Gymnastics Men's Horse Vault,Gymnastics,0,1,25.0,0.00496,Greece,Germany,[Men's],NaN
4,0,0.000000,1.239639,0.806687,Gymnastics Men's Parallel Bars,Gymnastics,0,1,25.0,0.00496,Greece,Germany,[Men's],NaN


In [ ]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# 정수형 데이터 정규화
scaler = MinMaxScaler()
integer_data = np.array([[10], [20], [30]])
scaled_data = scaler.fit_transform(integer_data)

# 문자열 데이터 원-핫 인코딩
encoder = OneHotEncoder(sparse=False)
categorical_data = np.array([['red'], ['blue'], ['green']])
encoded_data = encoder.fit_transform(categorical_data)

##모델 학습

옵션1. 트리 기반 모델로 이벤트별 메달 획득에 중요 영향을 미치는 특성 추출

옵션2. 신경망모델로 신체적 특성이 성과에 미치는 영향을 정량화, 특정 종목에서 이상적인 신체 조건 예측

###옵션1. 트리 모델

결측치행을 제거한 트리모델을 위한 데이터셋을 별도로 준비한다.

랜덤포레스트:

In [ ]:
# 가상의 데이터 로드 (올림픽 데이터셋 예시)
# 이 예시에서는 iris 데이터셋을 사용
# data = load_iris()
X = data.data
y = data.target
feature_names = data.feature_names

# 랜덤포레스트 모델 생성 및 훈련
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

# 특성 중요도 추출
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]  # 중요도가 높은 순서로 인덱스 정렬

# 중요도가 높은 상위 3개 특성 출력
print("Top 3 important features:")
for i in range(3):
    print(f"{i + 1}. feature {feature_names[indices[i]]} ({importances[indices[i]]:.2f})")


###옵션2. 신경망 모델

DNN을 위한 데이터 처리
* 결측치
* 이상치
*

In [ ]:
# 임의의 데이터 생성 (나이, 체중)
np.random.seed(0)
X_train = np.random.normal(30, 10, (100, 2))  # 평균 30, 표준편차 10의 정규분포
y_train_height = np.random.normal(170, 10, (100, ))  # 평균 170, 표준편차 10의 정규분포, 키
y_train_bmi = np.random.normal(25, 4, (100, ))  # 평균 25, 표준편차 4의 정규분포, BMI
y_train = np.column_stack((y_train_height, y_train_bmi))  # 키와 BMI를 하나의 배열로 결합

# 모델 정의
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(2,)),  # 입력 특성은 2개: 나이, 체중
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='linear')  # 출력층: 예측해야 할 키와 BMI, 선형 활성화 함수 사용
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X_train, y_train, epochs=10, verbose=1)